In [5]:
import networkx as nx
from networkx.algorithms import bipartite
import random

# Création du graphe bipartite
B = nx.Graph()
# Ajout des nœuds avec l'attribut 'bipartite' (0 pour étudiants, 1 pour clubs)
students = ["Alice", "Bob", "Charlie", "Dave"]
clubs = ["Club1", "Club2", "Club3"]
B.add_nodes_from(students, bipartite=0)
B.add_nodes_from(clubs, bipartite=1)

# Ajout des arêtes
edges = [("Alice", "Club1"), ("Alice", "Club2"), ("Bob", "Club1"),
         ("Charlie", "Club2"), ("Charlie", "Club3"), ("Dave", "Club3")]
B.add_edges_from(edges)


In [4]:
import networkx as nx
import random

# Calcul de la modularité bipartite
def compute_bipartite_modularity(B, partition_top, partition_bottom, top_nodes, bottom_nodes):
    m = B.number_of_edges()
    degree_top = {node: B.degree(node) for node in top_nodes}
    degree_bottom = {node: B.degree(node) for node in bottom_nodes}

    Q = 0
    for u in top_nodes:
        for v in bottom_nodes:
            A_uv = 1 if B.has_edge(u, v) else 0
            expected_edges = (degree_top[u] * degree_bottom[v]) / (2 * m)
            delta = 1 if partition_top[u] == partition_bottom[v] else 0
            Q += (A_uv - expected_edges) * delta

    return Q / (2 * m)

# Algorithme BRIM
def brim(B, top_nodes, bottom_nodes, max_iter=100):
    # Initialisation aléatoire des partitions
    partition_top = {node: random.randint(0, 1) for node in top_nodes}
    partition_bottom = {node: random.randint(0, 1) for node in bottom_nodes}

    for _ in range(max_iter):
        # Optimisation des partitions pour les top_nodes
        for u in top_nodes:
            best_modularity = -1
            best_partition = partition_top[u]
            for p in set(partition_bottom.values()):
                partition_top[u] = p
                modularity = compute_bipartite_modularity(B, partition_top, partition_bottom, top_nodes, bottom_nodes)
                if modularity > best_modularity:
                    best_modularity = modularity
                    best_partition = p
            partition_top[u] = best_partition

        # Optimisation des partitions pour les bottom_nodes
        for v in bottom_nodes:
            best_modularity = -1
            best_partition = partition_bottom[v]
            for p in set(partition_top.values()):
                partition_bottom[v] = p
                modularity = compute_bipartite_modularity(B, partition_top, partition_bottom, top_nodes, bottom_nodes)
                if modularity > best_modularity:
                    best_modularity = modularity
                    best_partition = p
            partition_bottom[v] = best_partition

    return partition_top, partition_bottom

# Création du graphe bipartite
B = nx.Graph()
# Ajout des nœuds avec l'attribut 'bipartite' (0 pour étudiants, 1 pour clubs)
students = ["Alice", "Bob", "Charlie", "Dave"]
clubs = ["Club1", "Club2", "Club3"]
B.add_nodes_from(students, bipartite=0)
B.add_nodes_from(clubs, bipartite=1)

# Ajout des arêtes
edges = [("Alice", "Club1"), ("Alice", "Club2"), ("Bob", "Club1"),
         ("Charlie", "Club2"), ("Charlie", "Club3"), ("Dave", "Club3")]
B.add_edges_from(edges)

# Identification des nœuds top et bottom
top_nodes = {n for n, d in B.nodes(data=True) if d['bipartite'] == 0}
bottom_nodes = set(B) - top_nodes

# Exécution de l'algorithme BRIM
partition_top, partition_bottom = brim(B, top_nodes, bottom_nodes)

print("Partitions des étudiants :", partition_top)
print("Partitions des clubs :", partition_bottom)


Partitions des étudiants : {'Alice': 0, 'Dave': 0, 'Bob': 0, 'Charlie': 0}
Partitions des clubs : {'Club1': 0, 'Club2': 0, 'Club3': 0}
